# PF400 Node Tests
## Setup and Basic Tests

In [ ]:
from madsci.client.node.rest_node_client import RestNodeClient
from madsci.common.types.action_types import ActionRequest
from madsci.common.types.location_types import LocationArgument

client = RestNodeClient(url="http://parker.cels.anl.gov:2002")

In [ ]:
client.get_status().description

In [ ]:
client.get_state()

## Test Data Setup

In [ ]:
# Test locations
source_loc = [102.162,26.226,67.451,712.056,71.524,995.235] #camera position
target_loc =  [164,58.489,86.752,661.513,122.221,-994.035]
rotation_loc =  [63,-27.659,113.485,631.997,72.163,994.458]
rotation_loc_resource_id = "01K7T1HRQQMXSB37SA5RGYFKMC"
# camera_loc = [90.597, 26.416, 66.422, 714.811, 81.916, 995.074]

# Approach locations (examples)
source_approach_single =  [166.87,0.648,98.948,706.188,70.855,995.16] # camera approach
source_approach_multiple = [
    [193.278,-4.397,72.689,735.146,70.85,995.233],
    source_approach_single,
]

In [ ]:
# ============================================
# Test Setup: Create Resources
# ============================================

from madsci.client.resource_client import ResourceClient
from madsci.common.types.resource_types import Asset, Slot

# Initialize resource client
resource_client = ResourceClient(resource_server_url="http://parker.cels.anl.gov:8004")

# Create source slot resource
# source_slot = Slot(
#     resource_name="test_source_slot",
#     resource_class="PlateSlot",
#     capacity=1,
#     attributes={
#         "description": "Test source slot for plate",
#     },
# )
# source_slot = resource_client.add_or_update_resource(source_slot)
# print(f"Source slot created: {source_slot.resource_id}")

# Create target slot resource
# target_slot = Slot(
#     resource_name="test_target_slot",
#     resource_class="PlateSlot",
#     capacity=1,
#     attributes={
#         "description": "Test target slot for plate",
#     },
# )
# target_slot = resource_client.add_or_update_resource(target_slot)
# print(f"Target slot created: {target_slot.resource_id}")

# Create plate asset resource with attributes
plate = resource_client.get_resource("01K7T1QAXCMSAJ3MAK2GAS24ZK")
plate.attributes = {
        "has_lid": True,
        "lid_height": 2.0,
        "grab_height_offset": 0.0,
        "description": "96-well microplate with lid",
    }
# plate = Asset(
#     resource_id="01K7T1QAXCMSAJ3MAK2GAS24ZK",
#     resource_name="sample_plate",
#     resource_class="Microplate",
#     attributes={
#         "has_lid": True,
#         "lid_height": 7.0,
#         "grab_height_offset": 20.0,
#         "description": "96-well microplate with lid",
#     },
# )
plate = resource_client.add_or_update_resource(plate)
print(f"Plate created with attributes: {plate.resource_id}")

# Push plate to source slot
# source_slot = resource_client.push(resource=source_slot.resource_id, child=plate)
# print(f"Plate pushed to source slot. Source quantity: {source_slot.quantity}")

# Store resource IDs for tests
# source_resource_id = source_slot.resource_id
# target_resource_id = target_slot.resource_id
source_resource_id = "01K7SZNCFGV604BD5C4ZFZQQW9"
target_resource_id = "01K7SZND789S1VN5VX77NKNMRX"


# print("\n=== Resource Setup Complete ===")
# print(f"Source Resource ID: {source_resource_id}")
# print(f"Target Resource ID: {target_resource_id}")
# print(f"Plate Resource ID: {plate.resource_id}")
# print(f"Plate Attributes: {plate.attributes}")

## Basic Tests 

In [ ]:
# Test 1: Basic transfer with old style (plain list locations)
request = ActionRequest(
    action_name="transfer",
    args={
        "source": LocationArgument(
            representation=source_loc,
            resource_id=source_resource_id,
        ).model_dump(mode="json"),
        "target": LocationArgument(
            representation=target_loc,
            resource_id=target_resource_id,
        ).model_dump(mode="json"),
    },
)
print("Test 1: Basic transfer")
response = client.send_action(action_request=request)

In [ ]:
client.get_action_result(response.action_id)

## New Feature Tests - Dictionary Representation with Rotation

In [ ]:
# Test 2: Transfer with rotation using dictionary representation
request = ActionRequest(
    action_name="transfer",
    args={
        "source": LocationArgument(
            representation={
                "location": source_loc,
                "plate_rotation": "narrow",
            },
            resource_id=source_resource_id,
        ).model_dump(mode="json"),
        "target": LocationArgument(
            representation={
                "location": target_loc,
                "plate_rotation": "wide",
                "press_depth": 5.0,
            },
            resource_id=target_resource_id,
        ).model_dump(mode="json"),
        "rotation_deck": LocationArgument(
            representation={
                "location": rotation_loc,
            },
            resource_id=rotation_loc_resource_id,
        ).model_dump(mode="json"),
    },
)
print("Test 2: Transfer with rotation (narrow -> wide)")
response = client.send_action(action_request=request)

In [ ]:
client.get_action_result(request.action_id)

In [ ]:
# Test 3: Transfer with None rotation (should default to narrow)
request = ActionRequest(
    action_name="transfer",
    args={
        "source": LocationArgument(
            representation={
                "location": source_loc,
                "plate_rotation": None,  # Will be treated as narrow
            },
            resource_id=source_resource_id,
        ).model_dump(mode="json"),
        "target": LocationArgument(
            representation={
                "location": target_loc,
                # No plate_rotation key - will be None
            },
            resource_id=target_resource_id,
        ).model_dump(mode="json"),
    },
)
print("Test 3: Transfer with None rotation (defaults to narrow)")
client.send_action(action_request=request)

In [ ]:
client.get_action_result(request.action_id)

## New Feature Tests - Single Approach Location

In [ ]:
# Test 4: Transfer with single approach location
request = ActionRequest(
    action_name="transfer",
    args={
        "source": LocationArgument(
            representation={
                "location": source_loc,
                "approach": source_approach_single,
                "plate_rotation": "narrow",
            },
            resource_id=source_resource_id,
        ).model_dump(mode="json"),
        "target": LocationArgument(
            representation={
                "location": target_loc,
                "plate_rotation": "wide",
            },
            resource_id=target_resource_id,
        ).model_dump(mode="json"),
        "rotation_deck": LocationArgument(
            representation={
                "location": rotation_loc,
            },
            resource_id=rotation_loc_resource_id,
        ).model_dump(mode="json"),
    },
)
print("Test 4: Transfer with single approach location")
client.send_action(action_request=request)

In [ ]:
client.get_action_result(request.action_id)

## New Feature Tests - Multiple Approach Locations

In [ ]:
# Test 5: Transfer with multiple approach locations
request = ActionRequest(
    action_name="transfer",
    args={
        "source": LocationArgument(
            representation={
                "location": source_loc,
                "approach": source_approach_multiple,  # List of lists
                "plate_rotation": "narrow",
            },
            resource_id=source_resource_id,
        ).model_dump(mode="json"),
        "target": LocationArgument(
            representation={
                "location": target_loc,
                # "approach": source_approach_multiple,  # Can have different approach for target
                "plate_rotation": "wide",
            },
            resource_id=target_resource_id,
        ).model_dump(mode="json"),
        "rotation_deck": LocationArgument(
            representation={
                "location": rotation_loc,
            },
            resource_id=rotation_loc_resource_id,
        ).model_dump(mode="json"),
    },
)
print("Test 5: Transfer with multiple approach locations")
client.send_action(action_request=request)

In [ ]:
client.get_action_result(request.action_id)

## New Feature Tests - Approach Height Offset

In [ ]:
# Test 6: Transfer with approach height offset
request = ActionRequest(
    action_name="transfer",
    args={
        "source": LocationArgument(
            representation={
                "location": source_loc,
                "approach_height_offset": 50.0,  # Add 5mm to default approach height
                "plate_rotation": "narrow",
            },
            resource_id=source_resource_id,
        ).model_dump(mode="json"),
        "target": LocationArgument(
            representation={
                "location": source_loc,
                "approach_height_offset": 10.0,  # Different offset for target
                "plate_rotation": "narrow",
            },
            resource_id=source_resource_id,
        ).model_dump(mode="json"),
    },
)
print("Test 6: Transfer with approach height offset")
client.send_action(action_request=request)

In [ ]:
client.get_action_result(request.action_id)

## New Feature Tests - Height Limit Validation

In [ ]:
# Test 7: Pick plate with height limit (should pass)
request = ActionRequest(
    action_name="pick_plate",
    args={
        "source": LocationArgument(
            representation={
                "location": source_loc,
                "height_limit": 60.0,  # High enough limit - should pass
            },
            resource_id=source_resource_id,
        ).model_dump(mode="json"),
    },
)
print("Test 7: Pick plate with valid height limit")
client.send_action(action_request=request)

In [ ]:
client.get_action_result(request.action_id)

In [ ]:
# Test 8: Pick plate with height limit (should fail)
request = ActionRequest(
    action_name="pick_plate",
    args={
        "source": LocationArgument(
            representation={
                "location": source_loc,
                # "approach_height_offset": 10.0,
                "height_limit": 60.0,  # Too low - should fail validation
            },
            resource_id=source_resource_id,
        ).model_dump(mode="json"),
    },
)
print("Test 8: Pick plate with invalid height limit (should fail)")
client.send_action(action_request=request)

In [ ]:
# This should show ActionFailed with height limit validation error
client.get_action_result(request.action_id)

## New Feature Tests - Combined Features

In [ ]:
# Test 9: Transfer with all features combined
request = ActionRequest(
    action_name="transfer",
    args={
        "source": LocationArgument(
            representation={
                "location": source_loc,
                "approach": source_approach_multiple,
                "plate_rotation": "narrow",
                "approach_height_offset": 15.0,
                "height_limit": 200.0,
            },
            resource_id=source_resource_id,
        ).model_dump(mode="json"),
        "target": LocationArgument(
            representation={
                "location": target_loc,
                "plate_rotation": "wide",
                "approach_height_offset": 50.0,
                "height_limit": 300.0,
            },
            resource_id=target_resource_id,
        ).model_dump(mode="json"),
        "rotation_deck": LocationArgument(
            representation={
                "location": rotation_loc,
            },
            resource_id=rotation_loc_resource_id,
        ).model_dump(mode="json"),
    },
)
print("Test 9: Transfer with all features combined")
client.send_action(action_request=request)

In [ ]:
client.get_action_result(request.action_id)

## Lid Operation Tests

In [ ]:
# Test 10: Remove lid with new features
request = ActionRequest(
    action_name="remove_lid",
    args={
        "source": LocationArgument(
            representation={
                "location": source_loc,
                "approach_height_offset": 15.0,
            },
            resource_id=source_resource_id,
        ).model_dump(mode="json"),
        "target": LocationArgument(
            representation={
                "location": rotation_loc,
            },
            resource_id=rotation_loc_resource_id,
        ).model_dump(mode="json"),
    },
)
print("Test 10: Remove lid with approach height offset")
client.send_action(action_request=request)

In [ ]:
client.get_action_result(request.action_id)

In [ ]:
# Test 11: Replace lid with new features
request = ActionRequest(
    action_name="replace_lid",
    args={
        "source": LocationArgument(
            representation={
                "location": rotation_loc,
            },
            resource_id=rotation_loc_resource_id,
        ).model_dump(mode="json"),
        "target": LocationArgument(
            representation={
                "location": source_loc,
                "approach_height_offset": 15.0,
            },
            resource_id=source_resource_id,
        ).model_dump(mode="json"),
    },
)
print("Test 11: Replace lid with approach height offset")
client.send_action(action_request=request)

In [ ]:
client.get_action_result(request.action_id)

## Place Plate Tests

In [ ]:
# Test 12: Place plate with all features
request = ActionRequest(
    action_name="place_plate",
    args={
        "target": LocationArgument(
            representation={
                "location": target_loc,
                "approach": source_approach_single,
                "plate_rotation": "wide",
                "approach_height_offset": 8.0,
                "height_limit": 400.0,
            },
            resource_id=target_resource_id,
        ).model_dump(mode="json"),
    },
)
print("Test 12: Place plate with all features")
client.send_action(action_request=request)

In [ ]:
client.get_action_result(request.action_id)

## Utility Commands

In [ ]:
# Check last action result
client.get_action_result(request.action_id)

In [ ]:
# Reset node
client.send_admin_command("reset")